In [1]:
!pip install -q ultralytics opencv-python pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.8 MB/s eta 0:00:00


In [3]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from google.colab import files

In [4]:
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

Saving dataset_video.mp4 to dataset_video.mp4


In [5]:
model = YOLO("yolov8n.pt")

100%|██████████| 6.25M/6.25M [00:00<00:00, 74.4MB/s]


In [6]:
def iou(box1, box2):
    x1, y1, x2, y2 = box1
    xx1, yy1, xx2, yy2 = box2
    xi1, yi1 = max(x1, xx1), max(y1, yy1)
    xi2, yi2 = min(x2, xx2), min(y2, yy2)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (xx2 - xx1) * (yy2 - yy1)
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area != 0 else 0

def are_close(bboxes, threshold=0.05):
    close_pairs = 0
    for i in range(len(bboxes)):
        for j in range(i+1, len(bboxes)):
            if iou(bboxes[i], bboxes[j]) > threshold:
                close_pairs += 1
    return close_pairs >= 3


In [7]:
cap = cv2.VideoCapture(video_path)
crowd_frames = []
frame_buffer = []
frame_number = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    results = model(frame, verbose=False)
    persons = []

    for r in results:
        for box, cls in zip(r.boxes.xyxy, r.boxes.cls):
            if int(cls) == 0:  # class 0 = person
                persons.append(box.tolist())

    if len(persons) >= 3 and are_close(persons):
        frame_buffer.append((frame_number, len(persons)))
        if len(frame_buffer) >= 10:
            crowd_frames.extend(frame_buffer)
            frame_buffer = []
    else:
        frame_buffer = []

    frame_number += 1

cap.release()
print("✅ Video processing complete.")


✅ Video processing complete.


In [8]:
df = pd.DataFrame(crowd_frames, columns=["frame_number", "person_count"])
df.to_csv("crowd_detection_output.csv", index=False)
print("📁 Output saved as: crowd_detection_output.csv")

📁 Output saved as: crowd_detection_output.csv


In [8]:
files.download("crowd_detection_output.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>